<a href="https://colab.research.google.com/github/SocSysSci/A6_Yakovenko/blob/main/A6_Yakovenko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 準備

### パッケージのインポート

In [ ]:
import random
import matplotlib.pyplot as plt

#### エージェントの定義

In [ ]:
class Agent:
  def __init__(self, initial_wealth, possible_dept = 0):
    self.wealth = initial_wealth
    self.possible_dept = possible_dept
    self.history = []
    self.history.append(self.wealth) # 初期資産の記録

  def register_to_history(self):
    self.history.append(self.wealth) # 現在の資産の記録

  def give_money_to(self, agent):
    if self.possible_dept < self.wealth:
      self.wealth -= 1
      agent.wealth += 1

#### エージェント間での資産のやりとりをする関数

In [ ]:
def transfer_money(agents):
  two_agents = random.sample(agents, 2)
  two_agents[0].give_money_to(two_agents[1])

## 2. シミュレーション

#### シミュレーション

In [ ]:
n = 500       # エージェント数
time = 5000   # シミュレーションの回数

# エージェントを生成
agents = [Agent(1) for _ in range(n)]

# シミュレーション本体
for i in range(time):
  # ランダムに選ばれた2エージェント間でお金をやりとり
  transfer_money(agents)
  # 全てのエージェントの資産を記録
  for a in agents:
    a.register_to_history()

# 資産の分布を表示
agent_wealth = [a.wealth for a in agents]
plt.hist(agent_wealth, bins=50)

#### 資産の変化を表示

In [ ]:
plt.figure(figsize=[20,5])
for a in agents:
  plt.plot(a.history)

## 3. ジニ係数

#### 関数の定義

In [ ]:
def gini(y):
  y.sort()
  y = [v - y[0] for v in y]
  n = len(y)
  nume = 0
  for i in range(n):
    nume += (i + 1) * y[i]
  deno = n * sum(y)
  g = ((2 * nume)/deno - (n+1)/n) * (n/(n-1))
  return g

#### ジニ係数の変化を見るシミュレーション

In [ ]:
n = 500      # エージェント数
time = 5000  # シミュレーション回数

# エージェントを生成
agents = [Agent(1) for _ in range(n)]

# シミュレーション本体
for i in range(time):
  transfer_money(agents)
  for a in agents:
    a.register_to_history()
  if i % 1000 == 0 or i == time-1: # 1000回ごとに表示
    agent_wealth = [a.wealth for a in agents]
    print("Time:{}, Gini:{}".format(i, gini(agent_wealth)))
    plt.hist(agent_wealth, bins=10)
    plt.show()


## 4. 会社モデル

In [ ]:
n = 500     # エージェント数
time = 5000 # シミュレーション回数

# 会社のパラメータ
K = 100 # 借入金（資本）
L = 20  # 社員数
W = 5   # 給料
R = 1   # 価格
r = 0.1 # 利子
b = 0.8 # 生産係数

# 製品の生産量を計算する関数
def number_of_product(worker_num, capital, beta):
  return int((worker_num ** beta) * (capital ** (1 - beta)))

# エージェントを生成
agents = [Agent(10) for _ in range(n)]

# シミュレーション本体
for i in range(time):
  # 製品数
  prod_num = number_of_product(L, K, b)

  # 社長の収入
  a = random.choice(agents)
  a.wealth = R * prod_num - L * W - r * K

  # 貸付した人の収入収入
  a = random.choice(agents)
  a.wealth = r * K

  # 社員の収入
  workers = random.sample(agents, L)
  for w in workers:
    w.wealth += W

  # 顧客（商品を買うために支払う）
  customers = random.sample(agents, prod_num)
  for c in customers:
    c.wealth -= R

  # 全員の資産を記録
  for a in agents:
    a.register_to_history()

# 資産の分布を表示
agent_wealth = [a.wealth for a in agents]
plt.hist(agent_wealth, bins=50)